# WEB SCRAPING

### Código utilizado durante el proceso de extracción de imágenes mediante web scraping

Web scrapeada: https://www.moto1pro.com/

In [2]:
from bs4 import BeautifulSoup
import requests
from lxml import html
from urllib.parse import urljoin
from selenium import webdriver
from urllib.request import urlretrieve
import time
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import warnings
import pandas as pd
from selenium.webdriver.support.ui import WebDriverWait   
from selenium.webdriver.support import expected_conditions as EC
import csv
warnings.filterwarnings('ignore')
options=Options()

options.add_extension('../driver/adblock.crx')
options.add_argument('--start-maximized')
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)
#options.add_argument('--incognito')
#options.add_argument('user-data-dir=selenium')

In [3]:
PATH = ChromeDriverManager().install()

In [4]:
driver = webdriver.Chrome(PATH, options=options)

In [5]:
url = 'https://www.moto1pro.com/'

In [6]:
driver.get(url)

In [7]:
# Acepto las cookies
cookies = driver.find_element(By.XPATH, '/html/body/div[11]/div[2]/div[1]/div[2]/div[2]/button[1]/p')
cookies.click()
time.sleep(0.5)

In [8]:
#Accedo a las fichas tecnicas
fichas = driver.find_elements(By.XPATH, '//*[@id="navWrapper"]/li[6]/a')
fichas[0].click()
time.sleep(0.5)

In [9]:
elems = driver.find_elements(By.XPATH, "//a[@href]")

In [10]:
links = []
for elem in elems:
    links.append(elem.get_attribute("href"))

In [11]:
links = [elem.get_attribute("href") for elem in elems]

In [12]:
links_marcas = []             
for e in links:
    if 'marca-de-motos/' in e:
        links_marcas.append(e)
#links_marcas

In [13]:
url_base = "https://www.moto1pro.com/marca-de-motos/"                                    
marcas = ['Aprilia', 'Bmw', 'Ducati', 'Honda', 'Yamaha', 'Kawasaki', 'Suzuki', 'KTM']

full_links = []                                                           
visited_links = set()

for marca in marcas:
    url = urljoin(url_base, marca)
    response = requests.get(url)
    tree = html.fromstring(response.content)
    div_content = tree.xpath('//div[@class="contenido-izquierda"]')[0]
    links = div_content.xpath('.//a/@href')
    for link in links:
        full_link = urljoin(url, link)
        if full_link not in visited_links:
            full_links.append(full_link)
            visited_links.add(full_link)
#full_links

In [14]:
url_base = "https://www.moto1pro.com/marca-de-motos"
marcas = ['Aprilia', 'Bmw', 'Ducati', 'Honda', 'Yamaha', 'Kawasaki', 'Suzuki', 'KTM']

image_links = []
for marca in marcas:
    full_link = urljoin(url_base, f"marca-de-motos/{marca}")
    r = requests.get(full_link)
    soup = BeautifulSoup(r.text, 'lxml')
    try:
        elements_with_image = soup.find_all('img', {'class': 'img-responsive'})
        for element in elements_with_image:
            image_links.append(element['src'])
    except:
        print(f"No se pudieron obtener los datos para la marca: {marca}")

df = pd.DataFrame({'image_links': image_links})

In [15]:
pd.set_option('display.max_colwidth', None)

In [16]:
df_images = df

In [17]:
df_images

,image_links
0,https://motos-b60.kxcdn.com/sites/default/files/aprilia-sr-gt-125-2022.jpg
1,https://motos-b60.kxcdn.com/sites/default/files/aprilia-tuono-660-factory.jpg
2,https://motos-b60.kxcdn.com/sites/default/files/aprilia-tuareg-660-evocative.jpg
3,https://motos-b60.kxcdn.com/sites/default/files/aprilia-tuareg-660.jpg
4,https://motos-b60.kxcdn.com/sites/default/files/aprilia-rx-125-2021.jpg
...,...
713,https://motos-b60.kxcdn.com/sites/default/files/KTM-1190-adveture-r-b.jpg
714,https://motos-b60.kxcdn.com/sites/default/files/timthumb_int_e137e728d58db641635454dd5fecceb5.timthumb.jpg
715,https://motos-b60.kxcdn.com/sites/default/files/ktm-duke-125.jpg
716,https://motos-b60.kxcdn.com/sites/default/files/ktm-rc-390-2016.jpg


In [71]:
#df_images.to_csv('image_URL.csv')